## Pure gemini model for input text

In [ ]:
%pip -q install -U google-generativeai gradio
import google.generativeai as genai, os
os.environ["GOOGLE_API_KEY"] = ""  # paste your key here just for testing
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# sanity check
print(genai.GenerativeModel("gemini-2.5-pro").generate_content("hi").text)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.2/315.2 kB 13.8 MB/s eta 0:00:00
Hello there! How can I help you today?


In [ ]:
# ==============================================================
# Stress Router + Gemini Chatbot with CSV Logging (turn-by-turn)
# ==============================================================

# 0) Setup
import os, re, csv, joblib
from datetime import datetime
from zoneinfo import ZoneInfo

# Try Colab Drive mount
try:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=False)
    DRIVE_MOUNTED = True
except Exception:
    DRIVE_MOUNTED = False
    print("Note: not in Colab. Skipping Drive mount.")

# 1) Paths
DATA_ROOT = "/content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final"
MODEL_DIR = "/content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final/models/combined_strat/best"
LOG_DIR   = os.path.join(DATA_ROOT, "logs")
os.makedirs(LOG_DIR, exist_ok=True)
LOG_CSV   = os.path.join(LOG_DIR, "svc_router_logs.csv")
print("Model dir:", MODEL_DIR)
print("Log CSV  :", LOG_CSV)

EN_SW = os.path.join(DATA_ROOT, "english_stopwords.txt")
CN_SW = os.path.join(DATA_ROOT, "chinese_stopwords.txt")

# 2) Recreate tokenizer env expected by the vectorizer
def load_stopwords(fp: str, lowercase=False) -> set:
    words = []
    try:
        with open(fp, "r", encoding="utf-8", errors="ignore") as f:
            for ln in f:
                ln = ln.strip()
                if ln and not ln.startswith("#"):
                    words.append(ln.lower() if lowercase else ln)
    except FileNotFoundError:
        words = [
            "the","a","an","and","or","of","to","in","for","on","at","with",
            "是","的","了","和","在","就","也","都"
        ]
    return set(words)

EN_STOP = load_stopwords(EN_SW, lowercase=True)
CN_STOP = load_stopwords(CN_SW, lowercase=False)

try:
    import jieba
    HAS_JIEBA = True
except Exception:
    HAS_JIEBA = False

CJK_RE = re.compile(r"[\u4e00-\u9fff]")
TOKEN_RE_EN = re.compile(r"[A-Za-z]+(?:'[A-Za-z]+)?")
FALLBACK_WORD_RE = re.compile(r"\w+")
NUM_PUNC_RE = re.compile(r"^[\W_]+$")

# Must be top-level name to satisfy unpickling
def mixed_tokenize(text: str):
    text = str(text).strip()
    toks = []

    # English
    en = [w.lower() for w in TOKEN_RE_EN.findall(text)]
    en = [w for w in en if w not in EN_STOP]
    toks.extend(en)

    # Chinese
    if CJK_RE.search(text):
        if HAS_JIEBA:
            cn = [w.strip() for w in jieba.cut(text, cut_all=False) if w.strip()]
        else:
            cn = [ch for ch in text if CJK_RE.match(ch)]
        cn = [w for w in cn if w not in CN_STOP and not NUM_PUNC_RE.match(w)]
        toks.extend(cn)

    # Fallback
    if not toks:
        toks = [t for t in FALLBACK_WORD_RE.findall(text.lower()) if t not in EN_STOP]
    return toks

# 3) Load saved artifacts
_VEC = None
_CLF = None
_THR = 0.45

def _load_threshold(thr_path: str, default_thr: float = 0.45) -> float:
    try:
        with open(thr_path, "r") as f:
            return float(f.read().strip())
    except Exception:
        return default_thr

def load_svc_model(model_dir: str):
    global _VEC, _CLF, _THR
    vec_fp = os.path.join(model_dir, "tfidf_vectorizer.joblib")
    clf_fp = os.path.join(model_dir, "classifier.joblib")
    thr_fp = os.path.join(model_dir, "inference_threshold.txt")

    if not os.path.exists(vec_fp):
        raise FileNotFoundError(f"Missing vectorizer at {vec_fp}")
    if not os.path.exists(clf_fp):
        raise FileNotFoundError(f"Missing classifier at {clf_fp}")

    _VEC = joblib.load(vec_fp)
    _CLF = joblib.load(clf_fp)
    _THR = _load_threshold(thr_fp, default_thr=0.45)

    if not hasattr(_CLF, "predict_proba"):
        raise RuntimeError("Classifier must be CalibratedClassifierCV to expose predict_proba.")

    print("Loaded vectorizer and classifier.")
    print(f"Decision threshold: {_THR}")

load_svc_model(MODEL_DIR)

# 4) Prediction helper that returns label and prob
def predict_with_prob(text_input: str):
    if _VEC is None or _CLF is None:
        load_svc_model(MODEL_DIR)
    X = _VEC.transform([text_input])
    prob = float(_CLF.predict_proba(X)[0, 1])
    is_stress = prob >= _THR
    print(f"[svc-v1] P(stress)={prob:.4f}, thr={_THR:.2f} -> {('STRESS' if is_stress else 'CALM')}")
    return bool(is_stress), prob

# 5) CSV logging (log BOTH user and assistant text)
def _ensure_csv_header(path: str):
    if not os.path.exists(path) or os.stat(path).st_size == 0:
        with open(path, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["timestamp", "prob", "label", "user_text", "assistant_text"])

def log_turn_to_csv(user_text: str, prob: float, is_stress: bool,
                    assistant_text: str, path: str = LOG_CSV):
    _ensure_csv_header(path)
    ts = datetime.now(ZoneInfo("America/Chicago")).isoformat(timespec="seconds")
    label = "STRESS" if is_stress else "CALM"

    # Keep one-row-per-turn
    safe_user_text = user_text.replace("\n", " ").strip()
    safe_assistant_text = assistant_text.replace("\n", " ").strip()

    with open(path, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([ts, round(prob, 4), label, safe_user_text, safe_assistant_text])

# 6) Gemini setup
import google.generativeai as genai

API_KEY = os.environ.get("GOOGLE_API_KEY")
if API_KEY is None:
    API_KEY = "PASTE_YOUR_GEMINI_API_KEY_HERE"
    if API_KEY == "PASTE_YOUR_GEMINI_API_KEY_HERE":
        raise ValueError("Please set GOOGLE_API_KEY in Colab Secrets or as an env var.")

genai.configure(api_key=API_KEY)

# This prompt is for when the user IS stressful
SYSTEM_PROMPT_STRESSFUL = """
**Your Role:** You are a compassionate, patient, and wise listening guide. Your goal is to help a user who is currently feeling stressed, anxious, or upset.

**Context:** The user's input has been identified as "stressful." Your first priority is to create a safe and calm space.

**Your 3-Step Process:**

**Step 1: Pacify (Acknowledge and Calm)**
* Immediately guide them with a simple, concrete calming exercise.
* Examples: "I hear you, that sounds very difficult. Before we talk, let's just take one deep breath together. Inhale slowly... and exhale." or "Thank you for sharing. That is a heavy feeling. Let's try to ground ourselves. Can you look around and name one thing you see that is blue?"
* After the exercise, acknowledge their feelings.

**Step 2: Analyze and Guide (The Six Categories)**
* Listen to their problem. Gently analyze if their suffering might be related to one of the following six unhelpful mind-states.
* Do not use the technical terms. Instead, identify the pattern and guide them away using the tools provided.

    * If craving: clinging, excessive desire, "I must have..."
        * Tool: use an analogy about impermanence or a verse about contentment.
    * If aversion or anger: blame, hatred, resentment.
        * Tool: use a story about forgiveness or an analogy like "holding anger is like holding a hot coal."
    * If confusion: feeling lost, no direction.
        * Tool: focus on truth and clarity, break the problem into small true pieces.
    * If comparison or conceit: "I am better" or "I am worse."
        * Tool: describe a beautiful scene (a forest where every tree is different but essential).
    * If doubt: paralyzing skepticism, lack of trust.
        * Tool: use a simple truth or verse, encourage one small step.
    * If rigid negative belief: "I am worthless."
        * Tool: use a story or rare event that gives a different perspective.

**Step 3: Conclude and Ask**
* After offering guidance, conclude with a supportive, open-ended question.
"""

# This prompt is for when the user is NOT stressful
SYSTEM_PROMPT_CALM = """
**Your Role:** You are a positive, encouraging, and wise companion.

**Context:** The user's input has been identified as calm. Your goal is to reinforce this positive state and provide tools for maintaining it.

**Your 3-Step Process:**

**Step 1: Compliment and Reinforce**
* Begin by genuinely acknowledging and complimenting their positive state.

**Step 2: Follow Their Interest**
* Follow their topics. Explore what they find interesting or joyful.

**Step 3: Conclude and Ask**
* Conclude with an open-ended question that invites further discussion.
"""

try:
    model_stress = genai.GenerativeModel(
        model_name="gemini-2.5-pro",
        system_instruction=SYSTEM_PROMPT_STRESSFUL
    )
    model_calm = genai.GenerativeModel(
        model_name="gemini-2.5-pro",
        system_instruction=SYSTEM_PROMPT_CALM
    )
    MODELS_LOADED = True
except Exception as e:
    MODELS_LOADED = False
    print(f"FATAL: could not initialize Gemini models: {e}")

# 7) Gradio app (messages-only for Gradio 6)
import gradio as gr

def _content_to_str(content):
    """Normalize Gradio Chatbot content (str, list, dict) to a plain string."""
    if isinstance(content, str):
        return content
    if isinstance(content, list):
        parts = []
        for c in content:
            if isinstance(c, dict) and "text" in c:
                parts.append(str(c["text"]))
            else:
                parts.append(str(c))
        return " ".join(parts)
    if isinstance(content, dict) and "text" in content:
        return str(content["text"])
    return str(content)

def convert_gradio_to_gemini(chat_history):
    """
    Gradio 6 Chatbot uses a list of dicts:
      {"role": "user" | "assistant", "content": <str or other>}
    Convert this into Gemini-style history with plain string parts.
    """
    if not chat_history:
        return []

    gemini_history = []
    for msg in chat_history:
        role = msg.get("role", "user")
        raw_content = msg.get("content", "")

        # Normalize to string
        content = _content_to_str(raw_content)

        # Strip router tag before sending to Gemini
        if isinstance(content, str):
            content = content.split("  [svc:", 1)[0]

        gemini_history.append({
            "role": "user" if role == "user" else "model",
            "parts": [content]
        })
    return gemini_history

SHOW_PROB = True

def respond(message, chat_history):
    # Gradio passes messages-format history (list[dict]) or None
    if chat_history is None:
        chat_history = []

    # Block empty messages
    if not message or not str(message).strip():
        return "", chat_history

    if not MODELS_LOADED:
        chat_history = list(chat_history)
        chat_history.append({"role": "user", "content": message})
        chat_history.append({
            "role": "assistant",
            "content": "Error: AI models could not be loaded. Check API key and configuration."
        })
        return "", chat_history

    # 1) Route via SVC
    is_stress, prob = predict_with_prob(message)

    tag = f"[svc: {'STRESS' if is_stress else 'CALM'}; p={prob:.2f}; thr={_THR:.2f}]"
    display_user = f"{message}  {tag}" if SHOW_PROB else f"{message}  [svc: {'STRESS' if is_stress else 'CALM'}]"

    # 2) Build Gemini history from existing messages
    gemini_history = convert_gradio_to_gemini(chat_history)
    chat_session = (model_stress if is_stress else model_calm).start_chat(history=gemini_history)

    # 3) Call Gemini
    try:
        response = chat_session.send_message(message)
        response_text = response.text
    except Exception as e:
        print("Gemini error:", e)
        response_text = f"Sorry, an error occurred when calling Gemini: {e}"

    # 4) Append new turn in messages format
    chat_history = list(chat_history)
    chat_history.append({"role": "user", "content": display_user})
    chat_history.append({"role": "assistant", "content": response_text})

    # 5) Log turn INCLUDING assistant text
    try:
        log_turn_to_csv(
            user_text=message,
            prob=prob,
            is_stress=is_stress,
            assistant_text=response_text,
            path=LOG_CSV
        )
    except Exception as e:
        print(f"[warn] logging failed: {e}")

    # Clear textbox, update chatbot
    return "", chat_history

def clear_chat():
    # For messages format, just return empty list
    return [], ""

with gr.Blocks() as demo:
    gr.Markdown("# 🧠 Compassionate AI Guide")
    gr.Markdown("Each turn is routed by your SVC classifier. Decisions are logged to CSV.")

    # Chatbot uses messages format by default in Gradio 6
    chatbot = gr.Chatbot(label="Chat", height=500)
    msg_box = gr.Textbox(label="Your message", placeholder="How are you feeling?")

    with gr.Row():
        send_btn = gr.Button("Send", variant="primary")
        clear_btn = gr.Button("Clear Chat")

    gr.Markdown(f"**Logging to:** `{LOG_CSV}`")

    # Enter key submits
    msg_box.submit(respond, [msg_box, chatbot], [msg_box, chatbot])

    # Send button submits
    send_btn.click(respond, [msg_box, chatbot], [msg_box, chatbot])

    # Clear button clears chat and textbox
    clear_btn.click(clear_chat, None, [chatbot, msg_box], queue=False)

print("Launching app...")
demo.launch(share=True, theme=gr.themes.Soft(), debug=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model dir: /content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final/models/combined_strat/best
Log CSV  : /content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final/logs/svc_router_logs.csv
Loaded vectorizer and classifier.
Decision threshold: 0.44999999999999996
Launching app...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://22790c7423ffcb6f51.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 2.041 seconds.
DEBUG:jieba:Loading model cost 2.041 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


[svc-v1] P(stress)=0.2789, thr=0.45 -> CALM
[svc-v1] P(stress)=0.2789, thr=0.45 -> CALM
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://22790c7423ffcb6f51.gradio.live


# Judge model edition

In [ ]:
# ==============================================================
# Stress Router + Gemini Chatbot with CSV Logging (turn-by-turn)
# With Gemini Judge fusion (50% SVC, 50% Judge)
# ==============================================================

# 0) Setup
import os, re, csv, joblib, json
from datetime import datetime
from zoneinfo import ZoneInfo

# Try Colab Drive mount
try:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=False)
    DRIVE_MOUNTED = True
except Exception:
    DRIVE_MOUNTED = False
    print("Note: not in Colab. Skipping Drive mount.")

# 1) Paths
DATA_ROOT = "/content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final"
MODEL_DIR = "/content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final/models/combined_strat/best"
LOG_DIR   = os.path.join(DATA_ROOT, "logs")
os.makedirs(LOG_DIR, exist_ok=True)
LOG_CSV   = os.path.join(LOG_DIR, "svc_router_logs.csv")
print("Model dir:", MODEL_DIR)
print("Log CSV  :", LOG_CSV)

EN_SW = os.path.join(DATA_ROOT, "english_stopwords.txt")
CN_SW = os.path.join(DATA_ROOT, "chinese_stopwords.txt")

# 2) Recreate tokenizer env expected by the vectorizer
def load_stopwords(fp: str, lowercase=False) -> set:
    words = []
    try:
        with open(fp, "r", encoding="utf-8", errors="ignore") as f:
            for ln in f:
                ln = ln.strip()
                if ln and not ln.startswith("#"):
                    words.append(ln.lower() if lowercase else ln)
    except FileNotFoundError:
        words = [
            "the","a","an","and","or","of","to","in","for","on","at","with",
            "是","的","了","和","在","就","也","都"
        ]
    return set(words)

EN_STOP = load_stopwords(EN_SW, lowercase=True)
CN_STOP = load_stopwords(CN_SW, lowercase=False)

try:
    import jieba
    HAS_JIEBA = True
except Exception:
    HAS_JIEBA = False

CJK_RE = re.compile(r"[\u4e00-\u9fff]")
TOKEN_RE_EN = re.compile(r"[A-Za-z]+(?:'[A-Za-z]+)?")
FALLBACK_WORD_RE = re.compile(r"\w+")
NUM_PUNC_RE = re.compile(r"^[\W_]+$")

# Must be top-level name to satisfy unpickling
def mixed_tokenize(text: str):
    text = str(text).strip()
    toks = []

    # English
    en = [w.lower() for w in TOKEN_RE_EN.findall(text)]
    en = [w for w in en if w not in EN_STOP]
    toks.extend(en)

    # Chinese
    if CJK_RE.search(text):
        if HAS_JIEBA:
            cn = [w.strip() for w in jieba.cut(text, cut_all=False) if w.strip()]
        else:
            cn = [ch for ch in text if CJK_RE.match(ch)]
        cn = [w for w in cn if w not in CN_STOP and not NUM_PUNC_RE.match(w)]
        toks.extend(cn)

    # Fallback
    if not toks:
        toks = [t for t in FALLBACK_WORD_RE.findall(text.lower()) if t not in EN_STOP]
    return toks

# 3) Load saved artifacts
_VEC = None
_CLF = None
_THR = 0.45

def _load_threshold(thr_path: str, default_thr: float = 0.45) -> float:
    try:
        with open(thr_path, "r") as f:
            return float(f.read().strip())
    except Exception:
        return default_thr

def load_svc_model(model_dir: str):
    global _VEC, _CLF, _THR
    vec_fp = os.path.join(model_dir, "tfidf_vectorizer.joblib")
    clf_fp = os.path.join(model_dir, "classifier.joblib")
    thr_fp = os.path.join(model_dir, "inference_threshold.txt")

    if not os.path.exists(vec_fp):
        raise FileNotFoundError(f"Missing vectorizer at {vec_fp}")
    if not os.path.exists(clf_fp):
        raise FileNotFoundError(f"Missing classifier at {clf_fp}")

    _VEC = joblib.load(vec_fp)
    _CLF = joblib.load(clf_fp)
    _THR = _load_threshold(thr_fp, default_thr=0.45)

    if not hasattr(_CLF, "predict_proba"):
        raise RuntimeError("Classifier must be CalibratedClassifierCV to expose predict_proba.")

    print("Loaded vectorizer and classifier.")
    print(f"Decision threshold: {_THR}")

load_svc_model(MODEL_DIR)

# 4) Prediction helper that returns label and prob
def predict_with_prob(text_input: str):
    if _VEC is None or _CLF is None:
        load_svc_model(MODEL_DIR)
    X = _VEC.transform([text_input])
    prob = float(_CLF.predict_proba(X)[0, 1])
    is_stress = prob >= _THR
    print(f"[svc-v1] P(stress)={prob:.4f}, thr={_THR:.2f} -> {('STRESS' if is_stress else 'CALM')}")
    return bool(is_stress), prob

# 5) CSV logging (now logs classifier, judge, fusion, reasoning)
def _ensure_csv_header(path: str):
    if not os.path.exists(path) or os.stat(path).st_size == 0:
        with open(path, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow([
                "timestamp",
                "clf_prob",
                "clf_label",
                "judge_prob",
                "judge_label",
                "judge_confidence",
                "judge_reasoning",
                "final_prob",
                "final_label",
                "user_text",
                "assistant_text",
            ])

def log_turn_to_csv(
    user_text: str,
    clf_prob: float,
    clf_label: str,
    judge_prob: float,
    judge_label: str,
    judge_confidence: str,
    judge_reasoning: str,
    final_prob: float,
    final_label: str,
    assistant_text: str,
    path: str = LOG_CSV,
):
    _ensure_csv_header(path)
    ts = datetime.now(ZoneInfo("America/Chicago")).isoformat(timespec="seconds")

    safe_user_text = user_text.replace("\n", " ").strip()
    safe_assistant_text = assistant_text.replace("\n", " ").strip()
    safe_judge_reason = judge_reasoning.replace("\n", " ").strip()

    with open(path, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([
            ts,
            round(clf_prob, 4),
            clf_label,
            round(judge_prob, 4),
            judge_label,
            judge_confidence,
            safe_judge_reason,
            round(final_prob, 4),
            final_label,
            safe_user_text,
            safe_assistant_text,
        ])

# 6) Gemini setup
import google.generativeai as genai

API_KEY = os.environ.get("GOOGLE_API_KEY")
if API_KEY is None:
    API_KEY = "PASTE_YOUR_GEMINI_API_KEY_HERE"
    if API_KEY == "PASTE_YOUR_GEMINI_API_KEY_HERE":
        raise ValueError("Please set GOOGLE_API_KEY in Colab Secrets or as an env var.")

genai.configure(api_key=API_KEY)

# Prompts
SYSTEM_PROMPT_STRESSFUL = """
**Your Role:** You are a compassionate, patient, and wise listening guide. Your goal is to help a user who is currently feeling stressed, anxious, or upset.

**Context:** The user's input has been identified as "stressful." Your first priority is to create a safe and calm space.

**Your 3-Step Process:**

**Step 1: Pacify (Acknowledge and Calm)**
* Immediately guide them with a simple, concrete calming exercise.
* Examples: "I hear you, that sounds very difficult. Before we talk, let's just take one deep breath together. Inhale slowly... and exhale." or "Thank you for sharing. That is a heavy feeling. Let's try to ground ourselves. Can you look around and name one thing you see that is blue?"
* After the exercise, acknowledge their feelings.

**Step 2: Analyze and Guide (The Six Categories)**
* Listen to their problem. Gently analyze if their suffering might be related to one of the following six unhelpful mind-states.
* Do not use the technical terms. Instead, identify the pattern and guide them away using the tools provided.

    * If craving: clinging, excessive desire, "I must have..."
        * Tool: use an analogy about impermanence or a verse about contentment.
    * If aversion or anger: blame, hatred, resentment.
        * Tool: use a story about forgiveness or an analogy like "holding anger is like holding a hot coal."
    * If confusion: feeling lost, no direction.
        * Tool: focus on truth and clarity, break the problem into small true pieces.
    * If comparison or conceit: "I am better" or "I am worse."
        * Tool: describe a beautiful scene (a forest where every tree is different but essential).
    * If doubt: paralyzing skepticism, lack of trust.
        * Tool: use a simple truth or verse, encourage one small step.
    * If rigid negative belief: "I am worthless."
        * Tool: use a story or rare event that gives a different perspective.

**Step 3: Conclude and Ask**
* After offering guidance, conclude with a supportive, open-ended question.
"""

SYSTEM_PROMPT_CALM = """
**Your Role:** You are a positive, encouraging, and wise companion.

**Context:** The user's input has been identified as calm. Your goal is to reinforce this positive state and provide tools for maintaining it.

**Your 3-Step Process:**

**Step 1: Compliment and Reinforce**
* Begin by genuinely acknowledging and complimenting their positive state.

**Step 2: Follow Their Interest**
* Follow their topics. Explore what they find interesting or joyful.

**Step 3: Conclude and Ask**
* Conclude with an open-ended question that invites further discussion.
"""

JUDGE_SYSTEM_PROMPT = """
You are a stress detection judge.
Given a user's message and another model's probability P(stress), you estimate:
- your own probability that the user is stressed,
- a label (STRESS or CALM),
- a confidence level,
- and a short reasoning step by step.

You MUST respond with a single line of valid JSON only, no extra text, with fields:
{
  "judge_prob": float between 0 and 1,
  "judge_label": "STRESS" or "CALM",
  "confidence": "low" or "medium" or "high",
  "reasoning": "short explanation within 50 words"
}

Use the user message content as the main evidence. Use the classifier probability as a weak prior only.
"""

try:
    model_stress = genai.GenerativeModel(
        model_name="gemini-2.5-pro",
        system_instruction=SYSTEM_PROMPT_STRESSFUL
    )
    model_calm = genai.GenerativeModel(
        model_name="gemini-2.5-pro",
        system_instruction=SYSTEM_PROMPT_CALM
    )
    model_judge = genai.GenerativeModel(
        model_name="gemini-2.5-pro",
        system_instruction=JUDGE_SYSTEM_PROMPT
    )
    MODELS_LOADED = True
except Exception as e:
    MODELS_LOADED = False
    print(f"FATAL: could not initialize Gemini models: {e}")

def call_judge_model(user_message: str, clf_prob: float, clf_label: str):
    """
    Ask Gemini judge to evaluate stress with probability and reasoning.
    Returns: (judge_prob, judge_label, confidence, reasoning)
    Fallbacks to classifier if parsing fails.
    """
    base_prompt = f"""
user_message: {user_message}
classifier_prob: {clf_prob:.4f}
classifier_label: {clf_label}
"""
    try:
        resp = model_judge.generate_content(base_prompt)
        txt = resp.text.strip()
        # Ensure we only parse the JSON part
        # Try to find first '{' and last '}'
        start = txt.find("{")
        end = txt.rfind("}")
        if start != -1 and end != -1 and end > start:
            txt = txt[start:end+1]
        data = json.loads(txt)

        judge_prob = float(data.get("judge_prob", clf_prob))
        judge_prob = max(0.0, min(1.0, judge_prob))  # clamp
        judge_label = data.get("judge_label", "STRESS" if judge_prob >= 0.5 else "CALM").upper()
        if judge_label not in ("STRESS", "CALM"):
            judge_label = "STRESS" if judge_prob >= 0.5 else "CALM"
        confidence = data.get("confidence", "medium").lower()
        if confidence not in ("low", "medium", "high"):
            confidence = "medium"
        reasoning = data.get("reasoning", "No reasoning provided.")
        return judge_prob, judge_label, confidence, reasoning

    except Exception as e:
        print("Judge parse error, falling back to classifier:", e)
        # Fallback to classifier opinion
        judge_prob = clf_prob
        judge_label = clf_label
        confidence = "low"
        reasoning = "Judge model output could not be parsed, so the classifier probability was used."
        return judge_prob, judge_label, confidence, reasoning

# 7) Gradio app (messages-only for Gradio 6)
import gradio as gr

def _content_to_str(content):
    """Normalize Gradio Chatbot content (str, list, dict) to a plain string."""
    if isinstance(content, str):
        return content
    if isinstance(content, list):
        parts = []
        for c in content:
            if isinstance(c, dict) and "text" in c:
                parts.append(str(c["text"]))
            else:
                parts.append(str(c))
        return " ".join(parts)
    if isinstance(content, dict) and "text" in content:
        return str(content["text"])
    return str(content)

def convert_gradio_to_gemini(chat_history):
    """
    Gradio 6 Chatbot uses a list of dicts:
      {"role": "user" | "assistant", "content": <str or other>}
    Convert this into Gemini-style history with plain string parts.
    """
    if not chat_history:
        return []

    gemini_history = []
    for msg in chat_history:
        role = msg.get("role", "user")
        raw_content = msg.get("content", "")

        # Normalize to string
        content = _content_to_str(raw_content)

        # Strip router tag before sending to Gemini
        if isinstance(content, str):
            content = content.split("  [svc:", 1)[0]

        gemini_history.append({
            "role": "user" if role == "user" else "model",
            "parts": [content]
        })
    return gemini_history

SHOW_PROB = True

def respond(message, chat_history):
    # Gradio passes messages-format history (list[dict]) or None
    if chat_history is None:
        chat_history = []

    # Block empty messages
    if not message or not str(message).strip():
        return "", chat_history

    if not MODELS_LOADED:
        chat_history = list(chat_history)
        chat_history.append({"role": "user", "content": message})
        chat_history.append({
            "role": "assistant",
            "content": "Error: AI models could not be loaded. Check API key and configuration."
        })
        return "", chat_history

    # 1) SVC classifier output
    is_stress_clf, clf_prob = predict_with_prob(message)
    clf_label = "STRESS" if is_stress_clf else "CALM"

    # 2) Gemini judge output
    judge_prob, judge_label, judge_conf, judge_reason = call_judge_model(
        user_message=message,
        clf_prob=clf_prob,
        clf_label=clf_label
    )
    print(f"[judge] P(stress)={judge_prob:.4f}, label={judge_label}, conf={judge_conf}")

    # 3) 50/50 fusion
    final_prob = 0.5 * clf_prob + 0.5 * judge_prob
    final_is_stress = final_prob >= 0.5
    final_label = "STRESS" if final_is_stress else "CALM"
    print(f"[fusion] final P(stress)={final_prob:.4f} -> {final_label}")

    # 4) Tag for display
    tag = (
        f"[svc={clf_label}; p_svc={clf_prob:.2f}; "
        f"judge={judge_label}; p_judge={judge_prob:.2f}; "
        f"p_final={final_prob:.2f}]"
    )
    display_user = f"{message}  {tag}" if SHOW_PROB else f"{message}  [final: {final_label}]"

    # 5) Build Gemini chat history and route with fused label
    gemini_history = convert_gradio_to_gemini(chat_history)
    routed_model = model_stress if final_is_stress else model_calm
    chat_session = routed_model.start_chat(history=gemini_history)

    # 6) Call conversation model
    try:
        response = chat_session.send_message(message)
        response_text = response.text
    except Exception as e:
        print("Gemini conversation error:", e)
        response_text = f"Sorry, an error occurred when calling the conversation model: {e}"

    # 7) Append to UI history
    chat_history = list(chat_history)
    chat_history.append({"role": "user", "content": display_user})
    chat_history.append({"role": "assistant", "content": response_text})

    # 8) Log everything including judge reasoning
    try:
        log_turn_to_csv(
            user_text=message,
            clf_prob=clf_prob,
            clf_label=clf_label,
            judge_prob=judge_prob,
            judge_label=judge_label,
            judge_confidence=judge_conf,
            judge_reasoning=judge_reason,
            final_prob=final_prob,
            final_label=final_label,
            assistant_text=response_text,
            path=LOG_CSV,
        )
    except Exception as e:
        print(f"[warn] logging failed: {e}")

    # Clear textbox, update chatbot
    return "", chat_history

def clear_chat():
    # For messages format, just return empty list
    return [], ""

with gr.Blocks() as demo:
    gr.Markdown("# 🧠 Compassionate AI Guide")
    gr.Markdown(
        "Each turn is classified by SVC, judged by Gemini, fused 50/50, "
        "then routed to a calm or stress-support style. All decisions are logged."
    )

    chatbot = gr.Chatbot(label="Chat", height=500)
    msg_box = gr.Textbox(label="Your message", placeholder="How are you feeling?")

    with gr.Row():
        send_btn = gr.Button("Send", variant="primary")
        clear_btn = gr.Button("Clear Chat")

    gr.Markdown(f"**Logging to:** `{LOG_CSV}`")

    msg_box.submit(respond, [msg_box, chatbot], [msg_box, chatbot])
    send_btn.click(respond, [msg_box, chatbot], [msg_box, chatbot])
    clear_btn.click(clear_chat, None, [chatbot, msg_box], queue=False)

print("Launching app...")
demo.launch(share=True, theme=gr.themes.Soft(), debug=True)


Mounted at /content/drive
Model dir: /content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final/models/combined_strat/best
Log CSV  : /content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final/logs/svc_router_logs.csv


/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")


Loaded vectorizer and classifier.
Decision threshold: 0.44999999999999996
Launching app...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a95b8f0dfb25c622a4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.621 seconds.
DEBUG:jieba:Loading model cost 0.621 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


[svc-v1] P(stress)=0.2789, thr=0.45 -> CALM
[judge] P(stress)=0.0100, label=CALM, conf=high
[fusion] final P(stress)=0.1445 -> CALM
[svc-v1] P(stress)=0.2789, thr=0.45 -> CALM
[judge] P(stress)=0.0200, label=CALM, conf=high
[fusion] final P(stress)=0.1495 -> CALM
[svc-v1] P(stress)=0.2789, thr=0.45 -> CALM
[judge] P(stress)=0.0100, label=CALM, conf=high
[fusion] final P(stress)=0.1445 -> CALM
[svc-v1] P(stress)=0.2789, thr=0.45 -> CALM
[judge] P(stress)=0.0100, label=CALM, conf=high
[fusion] final P(stress)=0.1445 -> CALM
[svc-v1] P(stress)=0.2007, thr=0.45 -> CALM
[judge] P(stress)=0.0100, label=CALM, conf=high
[fusion] final P(stress)=0.1054 -> CALM
[svc-v1] P(stress)=0.1964, thr=0.45 -> CALM
[judge] P(stress)=0.0100, label=CALM, conf=high
[fusion] final P(stress)=0.1032 -> CALM
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a95b8f0dfb25c622a4.gradio.live


In [ ]:
# ==============================================================
# Random Router + Gemini Chatbot with CSV Logging (turn by turn)
# No classifier, no judge - random calm or stress routing
# ==============================================================

# 0) Setup
import os, csv, random
from datetime import datetime
from zoneinfo import ZoneInfo

# Try Colab Drive mount
try:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=False)
    DRIVE_MOUNTED = True
except Exception:
    DRIVE_MOUNTED = False
    print("Note: not in Colab. Skipping Drive mount.")

# 1) Paths
DATA_ROOT = "/content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final"
LOG_DIR   = os.path.join(DATA_ROOT, "logs")
os.makedirs(LOG_DIR, exist_ok=True)

# Separate log file for the random router
LOG_CSV   = os.path.join(LOG_DIR, "random_router_logs_v2.csv")

print("Random Router Log CSV:", LOG_CSV)

# 2) CSV logging (routing only)
def _ensure_csv_header(path: str):
    if not os.path.exists(path) or os.stat(path).st_size == 0:
        with open(path, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow([
                "timestamp",
                "route_label",       # STRESS or CALM
                "route_prob",        # random score in [0, 1)
                "route_reasoning",   # explanation of the routing decision
                "user_text",
                "assistant_text",
            ])

def log_turn_to_csv(
    user_text: str,
    route_label: str,
    route_prob: float,
    route_reasoning: str,
    assistant_text: str,
    path: str = LOG_CSV,
):
    _ensure_csv_header(path)
    ts = datetime.now(ZoneInfo("America/Chicago")).isoformat(timespec="seconds")

    safe_user_text = user_text.replace("\n", " ").strip()
    safe_assistant_text = assistant_text.replace("\n", " ").strip()
    safe_route_reason = route_reasoning.replace("\n", " ").strip()

    with open(path, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([
            ts,
            route_label,
            round(route_prob, 4),
            safe_route_reason,
            safe_user_text,
            safe_assistant_text,
        ])

# 3) Gemini setup
import google.generativeai as genai

API_KEY = os.environ.get("GOOGLE_API_KEY")
if API_KEY is None:
    API_KEY = "PASTE_YOUR_GEMINI_API_KEY_HERE"
    if API_KEY == "PASTE_YOUR_GEMINI_API_KEY_HERE":
        raise ValueError("Please set GOOGLE_API_KEY in Colab Secrets or as an env var.")

genai.configure(api_key=API_KEY)

# Prompts
SYSTEM_PROMPT_STRESSFUL = """
**Your Role:** You are a compassionate, patient, and wise listening guide. Your goal is to help a user who is currently feeling stressed, anxious, or upset.

**Context:** The user's input has been identified as "stressful." Your first priority is to create a safe and calm space.

**Your 3-Step Process:**

**Step 1: Pacify (Acknowledge and Calm)**
* Immediately guide them with a simple, concrete calming exercise.
* Examples: "I hear you, that sounds very difficult. Before we talk, let's just take one deep breath together. Inhale slowly... and exhale." or "Thank you for sharing. That is a heavy feeling. Let's try to ground ourselves. Can you look around and name one thing you see that is blue?"
* After the exercise, acknowledge their feelings.

**Step 2: Analyze and Guide (The Six Categories)**
* Listen to their problem. Gently analyze if their suffering might be related to one of the following six unhelpful mind-states.
* Do not use the technical terms. Instead, identify the pattern and guide them away using the tools provided.
    * If craving: clinging, excessive desire, "I must have..."
        * Tool: use an analogy about impermanence or a verse about contentment.
    * If aversion or anger: blame, hatred, resentment.
        * Tool: use a story about forgiveness or an analogy like "holding anger is like holding a hot coal."
    * If confusion: feeling lost, no direction.
        * Tool: focus on truth and clarity, break the problem into small true pieces.
    * If comparison or conceit: "I am better" or "I am worse."
        * Tool: describe a beautiful scene (a forest where every tree is different but essential).
    * If doubt: paralyzing skepticism, lack of trust.
        * Tool: use a simple truth or verse, encourage one small step.
    * If rigid negative belief: "I am worthless."
        * Tool: use a story or rare event that gives a different perspective.

**Step 3: Conclude and Ask**
* After offering guidance, conclude with a supportive, open-ended question.
* It is ok to briefly share why you suggested a certain exercise or perspective, in friendly plain language.
"""

SYSTEM_PROMPT_CALM = """
**Your Role:** You are a positive, encouraging, and wise companion.

**Context:** The user's input has been identified as calm. Your goal is to reinforce this positive state and provide tools for maintaining it.

**Your 3-Step Process:**

**Step 1: Compliment and Reinforce**
* Begin by genuinely acknowledging and complimenting their positive state.

**Step 2: Follow Their Interest**
* Follow their topics. Explore what they find interesting or joyful.
* You can briefly explain why you think certain habits or ideas might help them keep this calm state.

**Step 3: Conclude and Ask**
* Conclude with an open-ended question that invites further discussion.
"""

try:
    model_stress = genai.GenerativeModel(
        model_name="gemini-2.5-pro",
        system_instruction=SYSTEM_PROMPT_STRESSFUL
    )
    model_calm = genai.GenerativeModel(
        model_name="gemini-2.5-pro",
        system_instruction=SYSTEM_PROMPT_CALM
    )
    MODELS_LOADED = True
except Exception as e:
    MODELS_LOADED = False
    print(f"FATAL: could not initialize Gemini models: {e}")

# 4) Gradio app
import gradio as gr

def _content_to_str(content):
    """Normalize Gradio Chatbot content (str, list, dict) to a plain string."""
    if isinstance(content, str):
        return content
    if isinstance(content, list):
        parts = []
        for c in content:
            if isinstance(c, dict) and "text" in c:
                parts.append(str(c["text"]))
            else:
                parts.append(str(c))
        return " ".join(parts)
    if isinstance(content, dict) and "text" in content:
        return str(content["text"])
    return str(content)

def convert_gradio_to_gemini(chat_history):
    """
    Gradio 6 Chatbot uses a list of dicts:
      {"role": "user" or "assistant", "content": <str or other>}
    Convert this into Gemini-style history with plain string parts.
    """
    if not chat_history:
        return []

    gemini_history = []
    for msg in chat_history:
        role = msg.get("role", "user")
        raw_content = msg.get("content", "")

        content = _content_to_str(raw_content)

        gemini_history.append({
            "role": "user" if role == "user" else "model",
            "parts": [content]
        })
    return gemini_history

SHOW_PROB = True  # show random routing score in tag

def respond(message, chat_history):
    if chat_history is None:
        chat_history = []

    # Block empty messages
    if not message or not str(message).strip():
        return "", chat_history, ""

    if not MODELS_LOADED:
        chat_history = list(chat_history)
        chat_history.append({"role": "user", "content": message})
        error_msg = "Error: AI models could not be loaded. Check API key and configuration."
        chat_history.append({"role": "assistant", "content": error_msg})
        return "", chat_history, "Routing information not available, models not loaded."

    # 1) Random routing decision
    route_prob = random.random()
    route_is_stress = route_prob >= 0.5
    route_label = "STRESS" if route_is_stress else "CALM"

    # 2) Tag for display on user bubble
    tag = f"[route={route_label}; p_route={route_prob:.2f}]"
    display_user = f"{message}  {tag}" if SHOW_PROB else f"{message}  [route: {route_label}]"

    # 3) Build Gemini chat history and route
    gemini_history = convert_gradio_to_gemini(chat_history)
    routed_model = model_stress if route_is_stress else model_calm
    chat_session = routed_model.start_chat(history=gemini_history)

    # 4) Call conversation model
    try:
        response = chat_session.send_message(message)
        response_text = response.text
    except Exception as e:
        print("Gemini conversation error:", e)
        response_text = f"Sorry, an error occurred when calling the conversation model: {e}"

    # 5) Routing reasoning
    style_word = "supportive, stress-focused" if route_is_stress else "calm, positive"
    route_reasoning = (
        f"The router randomly chose a {style_word} style because the random score "
        f"was {route_prob:.2f} with threshold 0.50, so the route label is {route_label}."
    )

    # Assistant message with embedded explanation
    assistant_display = (
        f"{response_text}\n\n"
        "---\n"
        f"*Routing explanation:* {route_reasoning}"
    )

    chat_history = list(chat_history)
    chat_history.append({"role": "user", "content": display_user})
    chat_history.append({"role": "assistant", "content": assistant_display})

    # 6) Log routing and conversation
    try:
        log_turn_to_csv(
            user_text=message,
            route_label=route_label,
            route_prob=route_prob,
            route_reasoning=route_reasoning,
            assistant_text=assistant_display,
            path=LOG_CSV,
        )
    except Exception as e:
        print(f"[warn] logging failed: {e}")

    # 7) Text for side panel
    side_panel_text = (
        f"Route label: {route_label} (random score {route_prob:.2f})\n"
        f"Reasoning: {route_reasoning}"
    )

    return "", chat_history, side_panel_text

def clear_chat():
    # Reset chatbot to initial greeting and reset input and side panel
    initial_history = [
        {"role": "assistant", "content": "Hello! Feel free to share anything!"}
    ]
    return initial_history, "", "Routing information will appear here after you send a message."

with gr.Blocks() as demo:
    gr.Markdown("# 🧠 Compassionate AI Guide (Random Router)")
    gr.Markdown(
        "Each turn is randomly routed (50/50) to a calm style or stress-support style response, "
        "then logged to CSV with routing information."
    )

    # Initial greeting in chat
    initial_chat = [
        {"role": "assistant", "content": "Hello! Feel free to share anything!"}
    ]
    chatbot = gr.Chatbot(label="Chat", height=500, value=initial_chat)

    msg_box = gr.Textbox(label="Your message", placeholder="How are you feeling?")

    with gr.Row():
        clear_btn = gr.Button("Clear Chat")
        send_btn = gr.Button("Send", variant="primary")  # send button on the right

    # Side panel for routing info
    side_md = gr.Markdown("Routing information will appear here after you send a message.")

    # Wire events
    msg_box.submit(respond, [msg_box, chatbot], [msg_box, chatbot, side_md])
    send_btn.click(respond, [msg_box, chatbot], [msg_box, chatbot, side_md])
    clear_btn.click(clear_chat, None, [chatbot, msg_box, side_md], queue=False)

print("Launching app...")
demo.launch(share=True, theme=gr.themes.Soft(), debug=True)


Mounted at /content/drive
Random Router Log CSV: /content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final/logs/random_router_logs_v2.csv
Launching app...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://48f5f49baf417bfcac.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Version 2 (send button adjusted, reasoning shown)

In [ ]:
# ==============================================================
# Stress Router + Gemini Chatbot with CSV Logging (turn-by-turn)
# With Gemini Judge fusion (50% SVC, 50% Judge)
# ==============================================================

# 0) Setup
import os, re, csv, joblib, json
from datetime import datetime
from zoneinfo import ZoneInfo

# Try Colab Drive mount
try:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=False)
    DRIVE_MOUNTED = True
except Exception:
    DRIVE_MOUNTED = False
    print("Note: not in Colab. Skipping Drive mount.")

# 1) Paths
DATA_ROOT = "/content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final"
MODEL_DIR = "/content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final/models/combined_strat/best"
LOG_DIR   = os.path.join(DATA_ROOT, "logs")
os.makedirs(LOG_DIR, exist_ok=True)
LOG_CSV   = os.path.join(LOG_DIR, "svc_router_logs.csv")
print("Model dir:", MODEL_DIR)
print("Log CSV  :", LOG_CSV)

EN_SW = os.path.join(DATA_ROOT, "english_stopwords.txt")
CN_SW = os.path.join(DATA_ROOT, "chinese_stopwords.txt")

# 2) Recreate tokenizer env expected by the vectorizer
def load_stopwords(fp: str, lowercase=False) -> set:
    words = []
    try:
        with open(fp, "r", encoding="utf-8", errors="ignore") as f:
            for ln in f:
                ln = ln.strip()
                if ln and not ln.startswith("#"):
                    words.append(ln.lower() if lowercase else ln)
    except FileNotFoundError:
        words = [
            "the","a","an","and","or","of","to","in","for","on","at","with",
            "是","的","了","和","在","就","也","都"
        ]
    return set(words)

EN_STOP = load_stopwords(EN_SW, lowercase=True)
CN_STOP = load_stopwords(CN_SW, lowercase=False)

try:
    import jieba
    HAS_JIEBA = True
except Exception:
    HAS_JIEBA = False

CJK_RE = re.compile(r"[\u4e00-\u9fff]")
TOKEN_RE_EN = re.compile(r"[A-Za-z]+(?:'[A-Za-z]+)?")
FALLBACK_WORD_RE = re.compile(r"\w+")
NUM_PUNC_RE = re.compile(r"^[\W_]+$")

# Must be top-level name to satisfy unpickling
def mixed_tokenize(text: str):
    text = str(text).strip()
    toks = []

    # English
    en = [w.lower() for w in TOKEN_RE_EN.findall(text)]
    en = [w for w in en if w not in EN_STOP]
    toks.extend(en)

    # Chinese
    if CJK_RE.search(text):
        if HAS_JIEBA:
            cn = [w.strip() for w in jieba.cut(text, cut_all=False) if w.strip()]
        else:
            cn = [ch for ch in text if CJK_RE.match(ch)]
        cn = [w for w in cn if w not in CN_STOP and not NUM_PUNC_RE.match(w)]
        toks.extend(cn)

    # Fallback
    if not toks:
        toks = [t for t in FALLBACK_WORD_RE.findall(text.lower()) if t not in EN_STOP]
    return toks

# 3) Load saved artifacts
_VEC = None
_CLF = None
_THR = 0.45

def _load_threshold(thr_path: str, default_thr: float = 0.45) -> float:
    try:
        with open(thr_path, "r") as f:
            return float(f.read().strip())
    except Exception:
        return default_thr

def load_svc_model(model_dir: str):
    global _VEC, _CLF, _THR
    vec_fp = os.path.join(model_dir, "tfidf_vectorizer.joblib")
    clf_fp = os.path.join(model_dir, "classifier.joblib")
    thr_fp = os.path.join(model_dir, "inference_threshold.txt")

    if not os.path.exists(vec_fp):
        raise FileNotFoundError(f"Missing vectorizer at {vec_fp}")
    if not os.path.exists(clf_fp):
        raise FileNotFoundError(f"Missing classifier at {clf_fp}")

    _VEC = joblib.load(vec_fp)
    _CLF = joblib.load(clf_fp)
    _THR = _load_threshold(thr_fp, default_thr=0.45)

    if not hasattr(_CLF, "predict_proba"):
        raise RuntimeError("Classifier must be CalibratedClassifierCV to expose predict_proba.")

    print("Loaded vectorizer and classifier.")
    print(f"Decision threshold: {_THR}")

load_svc_model(MODEL_DIR)

# 4) Prediction helper that returns label and prob
def predict_with_prob(text_input: str):
    if _VEC is None or _CLF is None:
        load_svc_model(MODEL_DIR)
    X = _VEC.transform([text_input])
    prob = float(_CLF.predict_proba(X)[0, 1])
    is_stress = prob >= _THR
    print(f"[svc-v1] P(stress)={prob:.4f}, thr={_THR:.2f} -> {('STRESS' if is_stress else 'CALM')}")
    return bool(is_stress), prob

# 5) CSV logging (now logs classifier, judge, fusion, reasoning)
def _ensure_csv_header(path: str):
    if not os.path.exists(path) or os.stat(path).st_size == 0:
        with open(path, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow([
                "timestamp",
                "clf_prob",
                "clf_label",
                "judge_prob",
                "judge_label",
                "judge_confidence",
                "judge_reasoning",
                "output_reasoning",      # new column for output model reasoning
                "final_prob",
                "final_label",
                "user_text",
                "assistant_text",
            ])

def log_turn_to_csv(
    user_text: str,
    clf_prob: float,
    clf_label: str,
    judge_prob: float,
    judge_label: str,
    judge_confidence: str,
    judge_reasoning: str,
    output_reasoning: str,    # new param
    final_prob: float,
    final_label: str,
    assistant_text: str,
    path: str = LOG_CSV,
):
    _ensure_csv_header(path)
    ts = datetime.now(ZoneInfo("America/Chicago")).isoformat(timespec="seconds")

    safe_user_text = user_text.replace("\n", " ").strip()
    safe_assistant_text = assistant_text.replace("\n", " ").strip()
    safe_judge_reason = judge_reasoning.replace("\n", " ").strip()
    safe_output_reason = output_reasoning.replace("\n", " ").strip()

    with open(path, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([
            ts,
            round(clf_prob, 4),
            clf_label,
            round(judge_prob, 4),
            judge_label,
            judge_confidence,
            safe_judge_reason,
            safe_output_reason,
            round(final_prob, 4),
            final_label,
            safe_user_text,
            safe_assistant_text,
        ])

# 6) Gemini setup
import google.generativeai as genai

API_KEY = os.environ.get("GOOGLE_API_KEY")
if API_KEY is None:
    API_KEY = "PASTE_YOUR_GEMINI_API_KEY_HERE"
    if API_KEY == "PASTE_YOUR_GEMINI_API_KEY_HERE":
        raise ValueError("Please set GOOGLE_API_KEY in Colab Secrets or as an env var.")

genai.configure(api_key=API_KEY)

# Prompts
SYSTEM_PROMPT_STRESSFUL = """
**Your Role:** You are a compassionate, patient, and wise listening guide. Your goal is to help a user who is currently feeling stressed, anxious, or upset.

**Context:** The user's input has been identified as "stressful." Your first priority is to create a safe and calm space.

**Your 3-Step Process:**

**Step 1: Pacify (Acknowledge and Calm)**
* Immediately guide them with a simple, concrete calming exercise.
* Examples: "I hear you, that sounds very difficult. Before we talk, let's just take one deep breath together. Inhale slowly... and exhale." or "Thank you for sharing. That is a heavy feeling. Let's try to ground ourselves. Can you look around and name one thing you see that is blue?"
* After the exercise, acknowledge their feelings.

**Step 2: Analyze and Guide (The Six Categories)**
* Listen to their problem. Gently analyze if their suffering might be related to one of the following six unhelpful mind-states.
* Do not use the technical terms. Instead, identify the pattern and guide them away using the tools provided.
    * If craving: clinging, excessive desire, "I must have..."
        * Tool: use an analogy about impermanence or a verse about contentment.
    * If aversion or anger: blame, hatred, resentment.
        * Tool: use a story about forgiveness or an analogy like "holding anger is like holding a hot coal."
    * If confusion: feeling lost, no direction.
        * Tool: focus on truth and clarity, break the problem into small true pieces.
    * If comparison or conceit: "I am better" or "I am worse."
        * Tool: describe a beautiful scene (a forest where every tree is different but essential).
    * If doubt: paralyzing skepticism, lack of trust.
        * Tool: use a simple truth or verse, encourage one small step.
    * If rigid negative belief: "I am worthless."
        * Tool: use a story or rare event that gives a different perspective.

**Step 3: Conclude and Ask**
* After offering guidance, conclude with a supportive, open-ended question.
* It is ok to briefly share why you suggested a certain exercise or perspective, in friendly plain language.
"""

SYSTEM_PROMPT_CALM = """
**Your Role:** You are a positive, encouraging, and wise companion.

**Context:** The user's input has been identified as calm. Your goal is to reinforce this positive state and provide tools for maintaining it.

**Your 3-Step Process:**

**Step 1: Compliment and Reinforce**
* Begin by genuinely acknowledging and complimenting their positive state.

**Step 2: Follow Their Interest**
* Follow their topics. Explore what they find interesting or joyful.
* You can briefly explain why you think certain habits or ideas might help them keep this calm state.

**Step 3: Conclude and Ask**
* Conclude with an open-ended question that invites further discussion.
"""

JUDGE_SYSTEM_PROMPT = """
You are a stress detection judge. Given a user's message and another model's probability P(stress), you estimate:
- your own probability that the user is stressed,
- a label (STRESS or CALM),
- a confidence level,
- and a short reasoning step by step.

You MUST respond with a single line of valid JSON only, no extra text, with fields:
{
  "judge_prob": float between 0 and 1,
  "judge_label": "STRESS" or "CALM",
  "confidence": "low" or "medium" or "high",
  "reasoning": "short explanation within 50 words in friendly plain language, suitable to show to the end user"
}

Use the user message content as the main evidence. Use the classifier probability as a weak prior only.
"""

try:
    model_stress = genai.GenerativeModel(
        model_name="gemini-2.5-pro",
        system_instruction=SYSTEM_PROMPT_STRESSFUL
    )
    model_calm = genai.GenerativeModel(
        model_name="gemini-2.5-pro",
        system_instruction=SYSTEM_PROMPT_CALM
    )
    model_judge = genai.GenerativeModel(
        model_name="gemini-2.5-pro",
        system_instruction=JUDGE_SYSTEM_PROMPT
    )
    MODELS_LOADED = True
except Exception as e:
    MODELS_LOADED = False
    print(f"FATAL: could not initialize Gemini models: {e}")

def call_judge_model(user_message: str, clf_prob: float, clf_label: str):
    """
    Ask Gemini judge to evaluate stress with probability and reasoning.
    Returns: (judge_prob, judge_label, confidence, reasoning)
    Fallbacks to classifier if parsing fails.
    """
    base_prompt = f"""
user_message: {user_message}
classifier_prob: {clf_prob:.4f}
classifier_label: {clf_label}
"""
    try:
        resp = model_judge.generate_content(base_prompt)
        txt = resp.text.strip()
        # Ensure we only parse the JSON part
        start = txt.find("{")
        end = txt.rfind("}")
        if start != -1 and end != -1 and end > start:
            txt = txt[start:end+1]
        data = json.loads(txt)

        judge_prob = float(data.get("judge_prob", clf_prob))
        judge_prob = max(0.0, min(1.0, judge_prob))  # clamp
        judge_label = data.get("judge_label", "STRESS" if judge_prob >= 0.5 else "CALM").upper()
        if judge_label not in ("STRESS", "CALM"):
            judge_label = "STRESS" if judge_prob >= 0.5 else "CALM"
        confidence = data.get("confidence", "medium").lower()
        if confidence not in ("low", "medium", "high"):
            confidence = "medium"
        reasoning = data.get("reasoning", "No reasoning provided.")
        return judge_prob, judge_label, confidence, reasoning

    except Exception as e:
        print("Judge parse error, falling back to classifier:", e)
        # Fallback to classifier opinion
        judge_prob = clf_prob
        judge_label = clf_label
        confidence = "low"
        reasoning = "Judge model output could not be parsed, so the classifier probability was used."
        return judge_prob, judge_label, confidence, reasoning

# 7) Gradio app (messages-only for Gradio 6)
import gradio as gr

def _content_to_str(content):
    """Normalize Gradio Chatbot content (str, list, dict) to a plain string."""
    if isinstance(content, str):
        return content
    if isinstance(content, list):
        parts = []
        for c in content:
            if isinstance(c, dict) and "text" in c:
                parts.append(str(c["text"]))
            else:
                parts.append(str(c))
        return " ".join(parts)
    if isinstance(content, dict) and "text" in content:
        return str(content["text"])
    return str(content)

def convert_gradio_to_gemini(chat_history):
    """
    Gradio 6 Chatbot uses a list of dicts:
      {"role": "user" | "assistant", "content": <str or other>}
    Convert this into Gemini-style history with plain string parts.
    """
    if not chat_history:
        return []

    gemini_history = []
    for msg in chat_history:
        role = msg.get("role", "user")
        raw_content = msg.get("content", "")

        # Normalize to string
        content = _content_to_str(raw_content)

        # Strip router tag before sending to Gemini
        if isinstance(content, str):
            content = content.split("  [svc:", 1)[0]

        gemini_history.append({
            "role": "user" if role == "user" else "model",
            "parts": [content]
        })
    return gemini_history

SHOW_PROB = True

def respond(message, chat_history):
    # Gradio passes messages-format history (list[dict]) or None
    if chat_history is None:
        chat_history = []

    # Block empty messages
    if not message or not str(message).strip():
        return "", chat_history, ""

    if not MODELS_LOADED:
        chat_history = list(chat_history)
        chat_history.append({"role": "user", "content": message})
        error_msg = "Error: AI models could not be loaded. Check API key and configuration."
        chat_history.append({
            "role": "assistant",
            "content": error_msg
        })
        return "", chat_history, "Judge reasoning not available, models not loaded."

    # 1) SVC classifier output
    is_stress_clf, clf_prob = predict_with_prob(message)
    clf_label = "STRESS" if is_stress_clf else "CALM"

    # 2) Gemini judge output
    judge_prob, judge_label, judge_conf, judge_reason = call_judge_model(
        user_message=message,
        clf_prob=clf_prob,
        clf_label=clf_label
    )
    print(f"[judge] P(stress)={judge_prob:.4f}, label={judge_label}, conf={judge_conf}")

    # 3) 50/50 fusion
    final_prob = 0.5 * clf_prob + 0.5 * judge_prob
    final_is_stress = final_prob >= 0.5
    final_label = "STRESS" if final_is_stress else "CALM"
    print(f"[fusion] final P(stress)={final_prob:.4f} -> {final_label}")

    # 4) Tag for display on user bubble
    tag = (
        f"[svc={clf_label}; p_svc={clf_prob:.2f}; "
        f"judge={judge_label}; p_judge={judge_prob:.2f}; "
        f"p_final={final_prob:.2f}]"
    )
    display_user = f"{message}  {tag}" if SHOW_PROB else f"{message}  [final: {final_label}]"

    # 5) Build Gemini chat history and route with fused label
    gemini_history = convert_gradio_to_gemini(chat_history)
    routed_model = model_stress if final_is_stress else model_calm
    chat_session = routed_model.start_chat(history=gemini_history)

    # 6) Call conversation model
    try:
        response = chat_session.send_message(message)
        response_text = response.text
    except Exception as e:
        print("Gemini conversation error:", e)
        response_text = f"Sorry, an error occurred when calling the conversation model: {e}"

    # 7) Output model reasoning (separate from judge_reasoning)
    style_word = "supportive, stress-focused" if final_is_stress else "calm, positive"
    output_reasoning = (
        f"I replied in a {style_word} style because the final stress probability "
        f"was {final_prob:.2f} (svc={clf_prob:.2f}, judge={judge_prob:.2f}) "
        f"and the fused label was {final_label}."
    )

    # Assistant message with embedded reasoning for the user
    assistant_display = (
        f"{response_text}\n\n"
        "---\n"
        f"*Model explanation:* {output_reasoning} "
        f"Judge reasoning: {judge_reason}"
    )

    chat_history = list(chat_history)
    chat_history.append({"role": "user", "content": display_user})
    chat_history.append({"role": "assistant", "content": assistant_display})

    # 8) Log everything including judge reasoning and output reasoning
    try:
        log_turn_to_csv(
            user_text=message,
            clf_prob=clf_prob,
            clf_label=clf_label,
            judge_prob=judge_prob,
            judge_label=judge_label,
            judge_confidence=judge_conf,
            judge_reasoning=judge_reason,
            output_reasoning=output_reasoning,
            final_prob=final_prob,
            final_label=final_label,
            assistant_text=assistant_display,
            path=LOG_CSV,
        )
    except Exception as e:
        print(f"[warn] logging failed: {e}")

    # 9) Prepare judge reasoning text to show in separate UI panel
    judge_ui_text = (
        f"Judge label: {judge_label} (p={judge_prob:.2f}, confidence={judge_conf})\n"
        f"Reasoning: {judge_reason}"
    )

    # Clear textbox, update chatbot and judge reasoning panel
    return "", chat_history, judge_ui_text

def clear_chat():
    # Reset chatbot to initial greeting and reset input and judge panel
    initial_history = [
        {"role": "assistant", "content": "Hello! Feel free to share anything!"}
    ]
    return initial_history, "", "Judge reasoning will appear here after you send a message."

with gr.Blocks() as demo:
    gr.Markdown("# 🧠 Compassionate AI Guide")
    gr.Markdown(
        "Each turn is classified by SVC, judged by Gemini, fused 50/50, "
        "then routed to a calm or stress-support style. All decisions are logged to CSV."
    )

    # Initial greeting in chat
    initial_chat = [
        {"role": "assistant", "content": "Hello! Feel free to share anything!"}
    ]
    chatbot = gr.Chatbot(label="Chat", height=500, value=initial_chat)

    msg_box = gr.Textbox(label="Your message", placeholder="How are you feeling?")

    with gr.Row():
        clear_btn = gr.Button("Clear Chat")
        send_btn = gr.Button("Send", variant="primary")  # send button on the right

    # Judge reasoning area under the buttons
    judge_md = gr.Markdown("Judge reasoning will appear here after you send a message.")

    # Wire events
    msg_box.submit(respond, [msg_box, chatbot], [msg_box, chatbot, judge_md])
    send_btn.click(respond, [msg_box, chatbot], [msg_box, chatbot, judge_md])
    clear_btn.click(clear_chat, None, [chatbot, msg_box, judge_md], queue=False)

print("Launching app...")
demo.launch(share=True, theme=gr.themes.Soft(), debug=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model dir: /content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final/models/combined_strat/best
Log CSV  : /content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final/logs/svc_router_logs.csv


/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")


Loaded vectorizer and classifier.
Decision threshold: 0.44999999999999996


TypeError: BlockContext.__init__() got an unexpected keyword argument 'theme'

In [ ]:
# ==============================================================
# Stress Router + Gemini Chatbot with CSV Logging (turn-by-turn)
# With Gemini Judge fusion (50% SVC, 50% Judge)
# ==============================================================

# 0) Setup
import os, re, csv, joblib, json
from datetime import datetime
from zoneinfo import ZoneInfo

# Try Colab Drive mount
try:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=False)
    DRIVE_MOUNTED = True
except Exception:
    DRIVE_MOUNTED = False
    print("Note: not in Colab. Skipping Drive mount.")

# 1) Paths
DATA_ROOT = "/content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final"
MODEL_DIR = "/content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final/models/combined_strat/best"
LOG_DIR   = os.path.join(DATA_ROOT, "logs")
os.makedirs(LOG_DIR, exist_ok=True)
LOG_CSV   = os.path.join(LOG_DIR, "svc_router_logs.csv")
print("Model dir:", MODEL_DIR)
print("Log CSV  :", LOG_CSV)

EN_SW = os.path.join(DATA_ROOT, "english_stopwords.txt")
CN_SW = os.path.join(DATA_ROOT, "chinese_stopwords.txt")

# 2) Recreate tokenizer env expected by the vectorizer
def load_stopwords(fp: str, lowercase=False) -> set:
    words = []
    try:
        with open(fp, "r", encoding="utf-8", errors="ignore") as f:
            for ln in f:
                ln = ln.strip()
                if ln and not ln.startswith("#"):
                    words.append(ln.lower() if lowercase else ln)
    except FileNotFoundError:
        words = [
            "the","a","an","and","or","of","to","in","for","on","at","with",
            "是","的","了","和","在","就","也","都"
        ]
    return set(words)

EN_STOP = load_stopwords(EN_SW, lowercase=True)
CN_STOP = load_stopwords(CN_SW, lowercase=False)

try:
    import jieba
    HAS_JIEBA = True
except Exception:
    HAS_JIEBA = False

CJK_RE = re.compile(r"[\u4e00-\u9fff]")
TOKEN_RE_EN = re.compile(r"[A-Za-z]+(?:'[A-Za-z]+)?")
FALLBACK_WORD_RE = re.compile(r"\w+")
NUM_PUNC_RE = re.compile(r"^[\W_]+$")

# Must be top level name to satisfy unpickling
def mixed_tokenize(text: str):
    text = str(text).strip()
    toks = []

    # English
    en = [w.lower() for w in TOKEN_RE_EN.findall(text)]
    en = [w for w in en if w not in EN_STOP]
    toks.extend(en)

    # Chinese
    if CJK_RE.search(text):
        if HAS_JIEBA:
            cn = [w.strip() for w in jieba.cut(text, cut_all=False) if w.strip()]
        else:
            cn = [ch for ch in text if CJK_RE.match(ch)]
        cn = [w for w in cn if w not in CN_STOP and not NUM_PUNC_RE.match(w)]
        toks.extend(cn)

    # Fallback
    if not toks:
        toks = [t for t in FALLBACK_WORD_RE.findall(text.lower()) if t not in EN_STOP]
    return toks

# 3) Load saved artifacts
_VEC = None
_CLF = None
_THR = 0.45

def _load_threshold(thr_path: str, default_thr: float = 0.45) -> float:
    try:
        with open(thr_path, "r") as f:
            return float(f.read().strip())
    except Exception:
        return default_thr

def load_svc_model(model_dir: str):
    global _VEC, _CLF, _THR
    vec_fp = os.path.join(model_dir, "tfidf_vectorizer.joblib")
    clf_fp = os.path.join(model_dir, "classifier.joblib")
    thr_fp = os.path.join(model_dir, "inference_threshold.txt")

    if not os.path.exists(vec_fp):
        raise FileNotFoundError(f"Missing vectorizer at {vec_fp}")
    if not os.path.exists(clf_fp):
        raise FileNotFoundError(f"Missing classifier at {clf_fp}")

    _VEC = joblib.load(vec_fp)
    _CLF = joblib.load(clf_fp)
    _THR = _load_threshold(thr_fp, default_thr=0.45)

    if not hasattr(_CLF, "predict_proba"):
        raise RuntimeError("Classifier must be CalibratedClassifierCV to expose predict_proba.")

    print("Loaded vectorizer and classifier.")
    print(f"Decision threshold: {_THR}")

load_svc_model(MODEL_DIR)

# 4) Prediction helper that returns label and prob
def predict_with_prob(text_input: str):
    if _VEC is None or _CLF is None:
        load_svc_model(MODEL_DIR)
    X = _VEC.transform([text_input])
    prob = float(_CLF.predict_proba(X)[0, 1])
    is_stress = prob >= _THR
    print(f"[svc-v1] P(stress)={prob:.4f}, thr={_THR:.2f} -> {('STRESS' if is_stress else 'CALM')}")
    return bool(is_stress), prob

# 5) CSV logging (now logs classifier, judge, fusion, reasoning)
def _ensure_csv_header(path: str):
    if not os.path.exists(path) or os.stat(path).st_size == 0:
        with open(path, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow([
                "timestamp",
                "clf_prob",
                "clf_label",
                "judge_prob",
                "judge_label",
                "judge_confidence",
                "judge_reasoning",
                "output_reasoning",
                "final_prob",
                "final_label",
                "user_text",
                "assistant_text",
            ])

def log_turn_to_csv(
    user_text: str,
    clf_prob: float,
    clf_label: str,
    judge_prob: float,
    judge_label: str,
    judge_confidence: str,
    judge_reasoning: str,
    output_reasoning: str,
    final_prob: float,
    final_label: str,
    assistant_text: str,
    path: str = LOG_CSV,
):
    _ensure_csv_header(path)
    ts = datetime.now(ZoneInfo("America/Chicago")).isoformat(timespec="seconds")

    safe_user_text = user_text.replace("\n", " ").strip()
    safe_assistant_text = assistant_text.replace("\n", " ").strip()
    safe_judge_reason = judge_reasoning.replace("\n", " ").strip()
    safe_output_reason = output_reasoning.replace("\n", " ").strip()

    with open(path, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([
            ts,
            round(clf_prob, 4),
            clf_label,
            round(judge_prob, 4),
            judge_label,
            judge_confidence,
            safe_judge_reason,
            safe_output_reason,
            round(final_prob, 4),
            final_label,
            safe_user_text,
            safe_assistant_text,
        ])

# 6) Gemini setup
import google.generativeai as genai

API_KEY = os.environ.get("GOOGLE_API_KEY")
if API_KEY is None:
    API_KEY = "PASTE_YOUR_GEMINI_API_KEY_HERE"
    if API_KEY == "PASTE_YOUR_GEMINI_API_KEY_HERE":
        raise ValueError("Please set GOOGLE_API_KEY in Colab Secrets or as an env var.")

genai.configure(api_key=API_KEY)

# Prompts
SYSTEM_PROMPT_STRESSFUL = """
**Your Role:** You are a compassionate, patient, and wise listening guide. Your goal is to help a user who is currently feeling stressed, anxious, or upset.

**Context:** The user's input has been identified as "stressful." Your first priority is to create a safe and calm space.

**Your 3-Step Process:**

**Step 1: Pacify (Acknowledge and Calm)**
* Immediately guide them with a simple, concrete calming exercise.
* Examples: "I hear you, that sounds very difficult. Before we talk, let's just take one deep breath together. Inhale slowly... and exhale." or "Thank you for sharing. That is a heavy feeling. Let's try to ground ourselves. Can you look around and name one thing you see that is blue?"
* After the exercise, acknowledge their feelings.

**Step 2: Analyze and Guide (The Six Categories)**
* Listen to their problem. Gently analyze if their suffering might be related to one of the following six unhelpful mind-states.
* Do not use the technical terms. Instead, identify the pattern and guide them away using the tools provided.
    * If craving: clinging, excessive desire, "I must have..."
        * Tool: use an analogy about impermanence or a verse about contentment.
    * If aversion or anger: blame, hatred, resentment.
        * Tool: use a story about forgiveness or an analogy like "holding anger is like holding a hot coal."
    * If confusion: feeling lost, no direction.
        * Tool: focus on truth and clarity, break the problem into small true pieces.
    * If comparison or conceit: "I am better" or "I am worse."
        * Tool: describe a beautiful scene (a forest where every tree is different but essential).
    * If doubt: paralyzing skepticism, lack of trust.
        * Tool: use a simple truth or verse, encourage one small step.
    * If rigid negative belief: "I am worthless."
        * Tool: use a story or rare event that gives a different perspective.

**Step 3: Conclude and Ask**
* After offering guidance, conclude with a supportive, open-ended question.
* It is ok to briefly share why you suggested a certain exercise or perspective, in friendly plain language.
"""

SYSTEM_PROMPT_CALM = """
**Your Role:** You are a positive, encouraging, and wise companion.

**Context:** The user's input has been identified as calm. Your goal is to reinforce this positive state and provide tools for maintaining it.

**Your 3-Step Process:**

**Step 1: Compliment and Reinforce**
* Begin by genuinely acknowledging and complimenting their positive state.

**Step 2: Follow Their Interest**
* Follow their topics. Explore what they find interesting or joyful.
* You can briefly explain why you think certain habits or ideas might help them keep this calm state.

**Step 3: Conclude and Ask**
* Conclude with an open-ended question that invites further discussion.
"""

JUDGE_SYSTEM_PROMPT = """
You are a stress detection judge. Given a user's message and another model's probability P(stress), you estimate:
- your own probability that the user is stressed,
- a label (STRESS or CALM),
- a confidence level,
- and a short reasoning step by step.

You MUST respond with a single line of valid JSON only, no extra text, with fields:
{
  "judge_prob": float between 0 and 1,
  "judge_label": "STRESS" or "CALM",
  "confidence": "low" or "medium" or "high",
  "reasoning": "short explanation within 50 words in friendly plain language, suitable to show to the end user"
}

Use the user message content as the main evidence. Use the classifier probability as a weak prior only.
"""

try:
    model_stress = genai.GenerativeModel(
        model_name="gemini-2.5-pro",
        system_instruction=SYSTEM_PROMPT_STRESSFUL
    )
    model_calm = genai.GenerativeModel(
        model_name="gemini-2.5-pro",
        system_instruction=SYSTEM_PROMPT_CALM
    )
    model_judge = genai.GenerativeModel(
        model_name="gemini-2.5-pro",
        system_instruction=JUDGE_SYSTEM_PROMPT
    )
    MODELS_LOADED = True
except Exception as e:
    MODELS_LOADED = False
    print(f"FATAL: could not initialize Gemini models: {e}")

# 7) Gradio app (messages-only for Gradio 6)
import gradio as gr

def _content_to_str(content):
    """Normalize Gradio Chatbot content (str, list, dict) to a plain string."""
    if isinstance(content, str):
        return content
    if isinstance(content, list):
        parts = []
        for c in content:
            if isinstance(c, dict) and "text" in c:
                parts.append(str(c["text"]))
            else:
                parts.append(str(c))
        return " ".join(parts)
    if isinstance(content, dict) and "text" in content:
        return str(content["text"])
    return str(content)

SHOW_PROB = True

def respond(message, chat_history):
    # Gradio passes messages-format history (list[dict]) or None
    if chat_history is None:
        chat_history = []

    # Block empty messages
    if not message or not str(message).strip():
        return "", chat_history, ""

    if not MODELS_LOADED:
        chat_history = list(chat_history)
        chat_history.append({"role": "user", "content": message})
        error_msg = "Error: AI models could not be loaded. Check API key and configuration."
        chat_history.append({
            "role": "assistant",
            "content": error_msg
        })
        return "", chat_history, "Judge reasoning not available, models not loaded."

    # 1) SVC classifier output
    is_stress_clf, clf_prob = predict_with_prob(message)
    clf_label = "STRESS" if is_stress_clf else "CALM"

    # 2) Gemini judge output
    judge_prob, judge_label, judge_conf, judge_reason = call_judge_model(
        user_message=message,
        clf_prob=clf_prob,
        clf_label=clf_label
    )
    print(f"[judge] P(stress)={judge_prob:.4f}, label={judge_label}, conf={judge_conf}")

    # 3) 50/50 fusion
    final_prob = 0.5 * clf_prob + 0.5 * judge_prob
    final_is_stress = final_prob >= 0.5
    final_label = "STRESS" if final_is_stress else "CALM"
    print(f"[fusion] final P(stress)={final_prob:.4f} -> {final_label}")

    # 4) Tag for display on user bubble
    tag = (
        f"[svc={clf_label}; p_svc={clf_prob:.2f}; "
        f"judge={judge_label}; p_judge={judge_prob:.2f}; "
        f"p_final={final_prob:.2f}]"
    )
    display_user = f"{message}  {tag}" if SHOW_PROB else f"{message}  [final: {final_label}]"

    # 5) Route to stress or calm model, with fresh history every time
    routed_model = model_stress if final_is_stress else model_calm
    chat_session = routed_model.start_chat(history=[])

    # 6) Call conversation model
    try:
        response = chat_session.send_message(message)
        response_text = response.text
    except Exception as e:
        print("Gemini conversation error:", e)
        response_text = f"Sorry, an error occurred when calling the conversation model: {e}"

    # 7) Output model reasoning (separate from judge_reasoning)
    style_word = "supportive, stress-focused" if final_is_stress else "calm, positive"
    output_reasoning = (
        f"I replied in a {style_word} style because the final stress probability "
        f"was {final_prob:.2f} (svc={clf_prob:.2f}, judge={judge_prob:.2f}) "
        f"and the fused label was {final_label}."
    )

    # Assistant message with embedded reasoning for the user
    assistant_display = (
        f"{response_text}\n\n"
        "---\n"
        f"*Model explanation:* {output_reasoning} "
        f"Judge reasoning: {judge_reason}"
    )

    chat_history = list(chat_history)
    chat_history.append({"role": "user", "content": display_user})
    chat_history.append({"role": "assistant", "content": assistant_display})

    # 8) Log everything including judge reasoning and output reasoning
    try:
        log_turn_to_csv(
            user_text=message,
            clf_prob=clf_prob,
            clf_label=clf_label,
            judge_prob=judge_prob,
            judge_label=judge_label,
            judge_confidence=judge_conf,
            judge_reasoning=judge_reason,
            output_reasoning=output_reasoning,
            final_prob=final_prob,
            final_label=final_label,
            assistant_text=assistant_display,
            path=LOG_CSV,
        )
    except Exception as e:
        print(f"[warn] logging failed: {e}")

    # 9) Prepare judge reasoning text to show in separate UI panel
    judge_ui_text = (
        f"Judge label: {judge_label} (p={judge_prob:.2f}, confidence={judge_conf})\n"
        f"Reasoning: {judge_reason}"
    )

    # Clear textbox, update chatbot and judge reasoning panel
    return "", chat_history, judge_ui_text

# 8) UI layout with warm theme passed at launch
warm_theme = gr.themes.Soft(
    primary_hue="yellow",   # golden
    secondary_hue="purple"  # saturated purple
)

with gr.Blocks() as demo:
    gr.Markdown("# 🧠 Compassionate AI Guide")
    gr.Markdown(
        "Each turn is classified by SVC, judged by Gemini, fused 50/50, "
        "then routed to a calm or stress-support style. All decisions are logged to CSV."
    )

    # Initial greeting
    initial_chat = [
        {"role": "assistant", "content": "Hello! Feel free to share anything!"}
    ]
    chatbot = gr.Chatbot(label="Chat", height=500, value=initial_chat)

    msg_box = gr.Textbox(label="Your message", placeholder="How are you feeling?")

    # Send button on the right using Columns
    with gr.Row():
        with gr.Column(scale=4):
            gr.HTML("")  # spacer
        with gr.Column(scale=1):
            send_btn = gr.Button("Send", variant="primary")

    # Judge reasoning panel
    judge_md = gr.Markdown("Judge reasoning will appear here after you send a message.")

    # Wiring
    msg_box.submit(respond, [msg_box, chatbot], [msg_box, chatbot, judge_md])
    send_btn.click(respond, [msg_box, chatbot], [msg_box, chatbot, judge_md])

print("Launching app...")
demo.launch(share=True, debug=True, theme=warm_theme)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model dir: /content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final/models/combined_strat/best
Log CSV  : /content/drive/My Drive/Rice University/25fall/ELEC509/Final Project/Dataset/Stress_final/logs/svc_router_logs.csv
Loaded vectorizer and classifier.
Decision threshold: 0.44999999999999996
Launching app...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5e1e1c6558c2ec547e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5e1e1c6558c2ec547e.gradio.live
